# Validating a classification model with Trubrics

In this tutorial of the [Titanic Use Case](https://www.kaggle.com/c/titanic), you will:
- Initialise a `DataContext` with ML datasets and metadata from the titanic use case
- Build some out-of-the-box validations on a trained model and the `DataContext` with the `ModelValidator`:
    - Performance validations:
        - With sklearn metrics
        - With custom metrics
        - On specific data sclices
    - Explainability validations (with permutation importance)
    - Minimum functionality validation
    - Inference time validation
- Build a custom **data validation**
- Save validations to a `Trubric`
- Execute the `Trubric` from file
- Execute the `Trubric` from the CLI tool

## Install the trubrics-sdk

In [ ]:
#! pip install trubrics

## Load data & model

In [ ]:
from trubrics.example import get_titanic_data_and_model
train_df, test_df, model = get_titanic_data_and_model()

## Init DataContext

In [ ]:
from trubrics.context import DataContext

In [ ]:
data_context = DataContext(
    name="my_first_dataset",
    version="0.1",
    target="Survived",
    testing_data=test_df,
    training_data=train_df,
)

## Init ModelValidator

In [ ]:
import rich  # for pretty prints

from trubrics.validations import ModelValidator
from examples.classification_titanic.custom_scorer import custom_scorers  # see ./custom_scorer.py example
from examples.classification_titanic.slicing_functions import slicing_functions  # see ./slicing_functions.py for examples

In [ ]:
model_validator = ModelValidator(
    data=data_context, model=model, custom_scorers=custom_scorers, slicing_functions=slicing_functions
)

## Use the ModelValidator to build out-of-the-box validations

In [ ]:
performance = [
    # validate overall model performance with a manual threshold
    model_validator.validate_performance_against_threshold(metric="recall", threshold=0.7),

    # validate model performance on a specific data slice
    model_validator.validate_performance_against_threshold(metric="recall", threshold=0.8, data_slice="children"),

    # validate model performance with a custom metric "my_custom_loss"
    model_validator.validate_performance_against_threshold(metric="my_custom_loss", threshold=-0.7),

    # validate model performance difference between train and test sets (overfit)
    model_validator.validate_performance_between_train_and_test(metric="recall", threshold=0.3),

    # validate model performance against an sklearn dummy model
    model_validator.validate_test_performance_against_dummy(metric="accuracy"),

    # validate model performance is regular between various data slices
    model_validator.validate_performance_std_across_slices(metric="accuracy", std_threshold=0.07, dataset="testing_data", data_slices=["male", "children"]),
]

for performance_val in performance:
    rich.print(performance_val.dict())

In [ ]:
explainability = [
    # validate that a feature is in the top N important features of permutation feature importances
    model_validator.validate_feature_in_top_n_important_features(dataset="testing_data", feature="Sex", top_n_features=3),

    # validate the top N features are the same in train and test sets (overfit)
    model_validator.validate_feature_importance_between_train_and_test(top_n_features=2),
]

for explainability_val in explainability:
    rich.print(explainability_val.dict())

Check out the docs for many more [out-of-the-box validations](https://trubrics.github.io/trubrics-sdk/validations/), such as:
- Test the inference time of your model
- Test your model's minimum functionality

## Build a custom validation

In [ ]:
from examples.classification_titanic.custom_validator import CustomValidator  # see custom_validator.py for example

In [ ]:
model_custom_validator = CustomValidator(data=data_context, model=model, custom_scorers=custom_scorers)

In [ ]:
custom = [
    model_custom_validator.validate_master_age(age_limit_master=13, severity="warning")
]

rich.print(custom[0].dict())

## Save validations as a trubric

In [ ]:
from trubrics.validations import Trubric

validations =  performance + explainability + custom

trubric = Trubric(
    name="model_validations",
    model_name="my_model",
    model_version="0.0.1",
    data_context_name=data_context.name,
    data_context_version=data_context.version,
    metadata={"some parameter": "xyz"},
    tags=["mars-demo"],
    validations=validations,
)

In [ ]:
# save trubric to a local .json
trubric.save_local(path="./my_first_trubric.json")

In [ ]:
# or save to trubrics UI
try:
    trubric.save_ui()
    print("Trubric saved to UI.")
except Exception as e:
    print(f"Error in saving to trubrics manager:\n{e}")

## Execute the trubric from file

The `TrubricRun` object allows you to pull in a Trubric from file, and run that directly against any model and DataContext.

In [ ]:
from trubrics.validations.run import TrubricRun

In [ ]:
trubric_from_file = Trubric.parse_file("./my_first_trubric.json")

trubric_run_context = TrubricRun(
    data_context=data_context,
    model=model,
    trubric=trubric_from_file,
    tags=["dev"],
    custom_validator=CustomValidator,
    custom_scorers=custom_scorers,
    slicing_functions=slicing_functions
)

In [ ]:
new_trubric = trubric_run_context.set_new_trubric()

In [ ]:
# new_trubric.save_local()  # save the new trubric
# new_trubric.save_ui()

## Execute a trubric from the CLI (our short CLI demo)

In [ ]:
! trubrics run \
  --no-save-ui \
  --run-context-path titanic-example-trubric \
  --trubric-output-file-path "my_new_trubric.json"

## Connection to the Trubrics platform

In [ ]:
import os
import getpass
os.environ["TRUBRICS_CONFIG_EMAIL"] = input("Enter your Trubrics account email:")
os.environ["TRUBRICS_CONFIG_PASSWORD"] = getpass.getpass("Enter your password:")

In [ ]:
! trubrics init --trubrics-user --project-name "demo project"

In [ ]:
! trubrics run \
  --save-ui \
  --run-context-path titanic-example-trubric \
  --trubric-output-file-path "my_new_trubric.json"